# Data curation

### Import some necessary functions

In [18]:
import os, pydicom, sys
import numpy as np
from Plot_And_Scroll_Images.Plot_Scroll_Images import plot_Image_Scroll_Bar_Image, plot_scroll_Image
import SimpleITK as sitk
from Image_Array_And_Mask_From_Dicom_RT import Dicom_to_Imagestack

## Finding the Data

### Find where we put our data

In [4]:
data_path = r'K:\Morfeus\BMAnderson\CNN\Data\Data_Liver\Liver_Disease_Ablation_Segmentation\Images'
print('We have ' + str(len(os.listdir(data_path))) + ' patients!')

We have 63 patients!


## Ensuring contour fidelity...

### Note that we've set 'get_images_mask' to False, this means we won't be getting any of the image data, just looking at the dicom RT files

In [5]:
Dicom_Reader = Dicom_to_Imagestack(get_images_mask=False)

In [6]:
Dicom_Reader.down_folder(data_path)

### What ROI names do we have?

#### This will tell us all the unique roi names, hence all_rois

In [7]:
for roi in Dicom_Reader.all_rois:
    print(roi)

GTV
Ablation
External
Liver_BMA_Program
Liver
Liver_Ethan
Liver_TriMesh
Liver_Ethan PV
Liver_Ethan PV_TriMesh
GTV_Expand_Hollow
Liver_MBS
Liver_MBS_TriMesh (8)
GTV 5mm
Excluded 5mm
GTV 8mm
Excluded 8mm
GTV 10mm
Excluded 10mm
Included
Liver_BMA_Program_4
Liver_BMA_Program_42
Liver_Ethan post-ablation
veins
Liver_MBS_Rigid
Liver_MBS_Elastic
Liver_MBS_TriMesh (4)
Liver_TriMesh (4)
Excluded 0mm
Liver_og
POI_1
Liver_Reduced
Needle_Track
Liver_after_MBS
Liver_MBS_TriMesh
test
Liver_MBS_TriMesh (12)
Liver_BMA_Program_2397
Liver_BMA_Program_2
Liver_w_veins
external
Liver_Ethan post ablation
Liver_BMA_Program_4_TriMesh
Liver_TriMesh (2)
Liver_Ethan post ablation PV
Liver_Ethan pre-ablation
test_TriMesh
Liver_Coarse
Needle
Liver_TriMesh (1)
GTV_og
Liver_BMA_Program_TriMesh
Liver_Ethan PV_TriMesh (6)
Liver_reduced
Liver_TriMesh (12)
Liver_MBS_2
GTV_CT 1->CT 14
Ablation_CT 14->CT 1
Ablation_test
Liver_BMA_Program_0
Liver_BMA_Program_1
Liver_Ethan pre ablation
Liver_Reduced_TriMesh
Liver_Ethan PV_T

## Make contour associations

#### We have quite a few contour names here.. now, we can either change the ROI names in the RT files, or make an associations file

#### The associations file associates a contour name with another one {'Current contour':'Desired name'}

In [8]:
associations = {'Liver_BMA_Program_4':'Liver',
                'bma_liver':'Liver',
                'best_liver':'Liver',
                'tried_liver':'Liver'}

### Tell the Dicom_Reader that we want to set the associations, get the images and mask for contour 'Liver'

In [9]:
Dicom_Reader.set_associations(associations)
Dicom_Reader.set_get_images_and_mask(True)
Dicom_Reader.set_contour_names(['liver'])

### Re-write RTs
#### This is commented out, because if I run it, then the example above won't show any different contour names

In [19]:
# Dicom_Reader.associations = associations
# for RT in Dicom_Reader.all_RTs:
#     Dicom_Reader.rewrite_RT(RT)

## Pulling images and mask

### We'll first do this with one patient

In [13]:
patient_data_path = os.path.join(data_path,os.listdir(data_path)[0],'CT 2')
Dicom_Reader.Make_Contour_From_directory(patient_data_path)
print('Done!')

Done!


## View images

In [14]:
%matplotlib inline

### The images and mask are saved within the Dicom_Reader class, so we just have to load them

In [15]:
Images = Dicom_Reader.ArrayDicom
mask = Dicom_Reader.mask # This is the mask

### Viewing nifti handles

In [20]:
dicom_handle = Dicom_Reader.dicom_handle
annotations_handle = Dicom_Reader.annotation_handle

### Threshold

In [16]:
Images[Images<-200] = -200
Images[Images>200] = 200

In [19]:
plot_Image_Scroll_Bar_Image(Images)

interactive(children=(IntSlider(value=0, description='Z'), Text(value='2D', description='view'), Output(layout…

In [14]:
Images[mask==1] += 300

## Recap

### Checking ROI contour names and making associations

### Loading in image and mask from desired contour name

### Viewing images and mask

## Separate into Train/Test/Validation

### This is also important, but I would recommend using the 'Parallel' approach available in https://github.com/brianmanderson/Dicom_Data_to_Numpy_Arrays